In [2]:
# DONE: Hello World
# DONE: Crawl over stored Files
# DONE: Operate Elastic Search Class
# DONE: Populate wiki_evidence
# DONE: How to manage a peristant volume
# TODOs: Information Retrieval program, given search and rank sentences.
# TODOs: Implement DELETE on Database Class
# https://stackoverflow.com/questions/34621093/persist-elastic-search-data-in-docker-container
# TODOs: Repurpose DrQA Elasticsearch Module
# TODOs: Run Full-Evidence

In [23]:
### CRAWL FILE-SYSTEM ###
import os
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

knowledge_path = "/Users/joshua.sheppard/extract_3"

def iter_filesys(path):
    if os.path.isfile(path):
        yield path

    elif os.path.isdir(path):
        for dir, _, filenames in os.walk(path):
                for f in filenames:
                    if not f.endswith('.DS_Store'):
                        yield os.path.join(dir, f)

    else:
        raise RuntimeError("Invalid path %s" % path)

kw_files = iter_filesys(knowledge_path)

kw_sample = []
for i in kw_files:
    kw_sample.append(i)

print(kw_sample, len(kw_sample))

['/Users/joshua.sheppard/extract_3/AC/wiki_73', '/Users/joshua.sheppard/extract_3/AC/wiki_87', '/Users/joshua.sheppard/extract_3/AC/wiki_80', '/Users/joshua.sheppard/extract_3/AC/wiki_74', '/Users/joshua.sheppard/extract_3/AC/wiki_89', '/Users/joshua.sheppard/extract_3/AC/wiki_42', '/Users/joshua.sheppard/extract_3/AC/wiki_45', '/Users/joshua.sheppard/extract_3/AC/wiki_11', '/Users/joshua.sheppard/extract_3/AC/wiki_16', '/Users/joshua.sheppard/extract_3/AC/wiki_29', '/Users/joshua.sheppard/extract_3/AC/wiki_20', '/Users/joshua.sheppard/extract_3/AC/wiki_27', '/Users/joshua.sheppard/extract_3/AC/wiki_18', '/Users/joshua.sheppard/extract_3/AC/wiki_44', '/Users/joshua.sheppard/extract_3/AC/wiki_88', '/Users/joshua.sheppard/extract_3/AC/wiki_43', '/Users/joshua.sheppard/extract_3/AC/wiki_75', '/Users/joshua.sheppard/extract_3/AC/wiki_81', '/Users/joshua.sheppard/extract_3/AC/wiki_86', '/Users/joshua.sheppard/extract_3/AC/wiki_72', '/Users/joshua.sheppard/extract_3/AC/wiki_26', '/Users/josh

In [24]:
from elastic_db import ElasticDB

# Params
PORT = "http://localhost:9200"
INDEX = "wiki_evidence"
DOC = "evidence"

# Init Elasticsearch DB
wiki_ev = ElasticDB(elastic_port=PORT, elastic_index=INDEX, elastic_doc=DOC)

INFO:elastic_db:Connecting to http://localhost:9200


In [25]:
import json

def get_contents(filename):
    """Parse the contents of a file. Each line is a JSON encoded document."""
    documents = []
    with open(filename) as f:
        for line in f:
            # Parse document
            doc = json.loads(line)
            if doc["text"] == "":
                continue
            if not doc:
                continue

            # Add Document
            # TODOs: Normalisation
            documents.append((doc['id'], doc["title"], doc['text']))

    return documents

sample = kw_sample[0:1][0]
#print(sample)

get_contents(sample)

[('8032909',
  'Kristian Bach Bak',
  'Kristian Bach Bak (né Bak Nielsen; born 20 October 1982) is a Danish former professional footballer. Bak usually played centre-back, but he has a past as a striker. He has played one game for the Danish under-21 national team.\nClub career.\nMidtjylland.\nBorn in Kibæk, Herning Municipality, Bak\'s first football club was Kibæk IF, but he already started began playing in the youth department of Ikast fS at age 16. Ikast fS later merged with Herning Fremad and created FC Midtjylland.\nBak started his senior career with Midtjylland in 2000. He helped the club win the 1999–2000 Danish 1st Division and gain promotion to the Danish Superliga. In his first years at the club, he played as a striker, but eventually settled as a centre-back.\nHe played all Midtjylland\'s matches during the 2003–04 Danish Superliga season and was rewarded with a new contract in November 2004, keeping him at the club until 2007. In 2005, he became the club\'s captain.\nHeere

In [26]:
#from multiprocessing import Pool as ProcessPool
from tqdm import tqdm

#workers = ProcessPool(3)

files = [f for f in kw_sample][0:1]
count = 0
with tqdm(total=len(files)) as pbar:
    for i in files:
        ev = get_contents(i)
        for j in ev:
            count += 1
            _id, title, text = j
            doc = {"id": _id, "title": title, "text": text}
            wiki_ev.add_doc(doc)

    pbar.update()

logger.info('Read %d docs.' % count)

  0%|          | 0/1 [00:00<?, ?it/s]INFO:elastic_transport.transport:POST http://localhost:9200/wiki_evidence/_doc [status:201 duration:1.130s]
INFO:elastic_db:Added document id BlbCwYIBMr6oNIdkePmW
INFO:elastic_transport.transport:POST http://localhost:9200/wiki_evidence/_doc [status:201 duration:0.007s]
INFO:elastic_db:Added document id B1bCwYIBMr6oNIdkevlC
INFO:elastic_transport.transport:POST http://localhost:9200/wiki_evidence/_doc [status:201 duration:0.007s]
INFO:elastic_db:Added document id CFbCwYIBMr6oNIdkevlJ
INFO:elastic_transport.transport:POST http://localhost:9200/wiki_evidence/_doc [status:201 duration:0.008s]
INFO:elastic_db:Added document id CVbCwYIBMr6oNIdkevlS
INFO:elastic_transport.transport:POST http://localhost:9200/wiki_evidence/_doc [status:201 duration:0.008s]
INFO:elastic_db:Added document id ClbCwYIBMr6oNIdkevlb
INFO:elastic_transport.transport:POST http://localhost:9200/wiki_evidence/_doc [status:201 duration:0.006s]
INFO:elastic_db:Added document id C1bCwY

In [27]:
### QUERY DB ###
import elastic_db
results = wiki_ev.search("the government, emails, national security")
results

INFO:elastic_transport.transport:POST http://localhost:9200/wiki_evidence/_search [status:200 duration:0.035s]


([{'_index': 'wiki_evidence',
   '_id': 'RVbCwYIBMr6oNIdkfPnp',
   '_score': 10.010617,
   '_ignored': ['text.keyword'],
   '_source': {'id': '8033463',
    'title': 'Frank Jimenez',
    'text': "Frank Ruben Jimenez (born November 8, 1964) became the 21st General Counsel of the U.S. Department of the Navy on September 25, 2006, following his nomination by George W. Bush and confirmation by the United States Senate. Jimenez served at the Navy after the change in administrations on January 20, 2009 until April 30, 2009 when he resigned to become Vice President and General Counsel at ITT Corporation.\nAs the General Counsel of the Navy, Jimenez managed over 600 attorneys worldwide, helped to oversee the Naval Criminal Investigative Service, and advised senior Navy and Marine Corps officials on litigation, acquisition, fiscal, environmental, property, personnel, legislative, and ethics issues.\nEarly life and career.\nJimenez, the son of Cuban immigrants Frank and Daisy Jimenez, grew up in